<a href="https://colab.research.google.com/github/smaliyu/AfriNLP/blob/main/yoruba_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training  a Base Model for Hate Detection in Tweets

In this notebook, I attempt to train a model that detects hate speech in tweets.

In [1]:
# mount drive to persist output
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# change working directory to drive
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
# clone the repo for access to the datasets and content
!git clone https://github.com/smaliyu/AfriNLP.git

In [4]:
# change working directory to the root directory of repo
%cd AfriNLP

/content/drive/MyDrive/AfriNLP


In [5]:
!pwd

/content/drive/MyDrive/AfriNLP


In [12]:
# necessary imports

import torch
import pandas as pd
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm


# Custom dataset class to prepare dataset for the encoder model

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)




In [8]:
# Load the dataset
data = pd.read_csv('datasets/yoruba.csv')

# Extract texts and labels
texts = data['tweet'].tolist()
labels = data['label'].tolist()

# Label Encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)



In [9]:
data.head()

,Unnamed: 0,id,tweet,label
0,0,78293806,"""@username àwon to bi e ni retard 🤣🤣😭😭😭 omo a...",Offensive
1,1,78293807,"""@username oponu, didirin di robo, ode jati ja...",Offensive
2,2,78293808,"""werey yin ti podo re 😂 ##url",Offensive
3,3,78293809,"""koniragba fun gbogbo awon werey ton beyin ati...",Offensive
4,4,78293810,"""ònpè mi ni zaddy! taloni zaddy weyrey 😂 k...",Offensive


In [10]:
# To see the mapping of encoded labels to original labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'Hate': 0, 'Normal': 1, 'Offensive': 2}


In [13]:
# Tokenization
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
max_length = 128
# Split the data into train and test sets first
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, encoded_labels, random_state=42, test_size=0.2)

# Now tokenize each set
def tokenize(texts):
    return tokenizer(texts, add_special_tokens=True, max_length=max_length, padding='max_length', return_attention_mask=True, truncation=True)

train_encodings = tokenize(train_texts)
test_encodings = tokenize(test_texts)

# Create Custom Dataset
train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Model and Optimizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=3)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
model.to(device)
# Training loop
num_epochs = 5

for epoch in tqdm(range(num_epochs)):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        optimizer.zero_grad()

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Training Loss: {avg_train_loss}")

# Evaluation
model.eval()
predictions = []

with torch.inference_mode():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.cpu().numpy())

accuracy = accuracy_score(test_labels, predictions)
print(f"Test Accuracy: {accuracy}")


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5, Average Training Loss: 0.6618851302003088
Epoch 2/5, Average Training Loss: 0.5191852412665421
Epoch 3/5, Average Training Loss: 0.4432771798630475
Epoch 4/5, Average Training Loss: 0.4089667884565075
Epoch 5/5, Average Training Loss: 0.3764025501620311
Test Accuracy: 0.8275862068965517


In [14]:
# Save only the model's state dictionary
torch.save(model.state_dict(), 'yoruba_base_model_state_dict.pth')